In [1]:
from matplotlib import rcParams
import matplotlib.pyplot as plt
import numpy as np

np.seterr(all='ignore')
rcParams['figure.figsize'] = (14, 6)

#扩展MOD
from funcat import * #FUNCAT的同花顺/通达信指标模块：和：&，或：|，其他基本相同。可以使用指标选股！！！
#from pytdx import * #通达信数据更新模块到QUANTAXIS的mongoDB
from tushare import * #TUSHARE数据连接更新。
from rqalpha import * #RQALPHA本地数据连接，通过 rqalpha update_bundle 8点后更新全部数据库
#from jaqs import * #QunatOS的jaqs客户端在线。
#from abupy import * #ABUPY本地化，数据在线下载更新，数据在线下载在根目录abu下
#from quantaxis import * #加载QUANTAXIS数据库模块，首先需要开启mongoDB+web服务器，数据更新通过通达信服务器。
#from hikyuu import * #HIKYUU本地化，数据库在根目录的stock。通过通达信导入。

#读取/设定数据库时间区域，当前股票，选股时间段。 
import time
import datetime
n1 = (time.strftime("20%y-%m-%d"))## 20yy-mm-dd格式，输出当日日期
print ("读取当日时间格式n1",n1) #当日时间格式：yyyy-mm-dd
n2 = (time.strftime("20%y%m%d"))## 20yymmdd格式，输出当日日期
print ("读取当日时间格式n2",n2) #当日时间格式：yyyymmdd
def getYesterday(): 
    today=datetime.date.today() 
    oneday=datetime.timedelta(days=10) 
    yesterday=today-oneday  
    return yesterday
n3 = (getYesterday())# 输出前一天日期
print ("读取前一日时间格式n3",n3) #前日时间格式：yyyy-mm-dd
i = datetime.datetime.now()#当前日期
n4 = ("%s%s%s" % (i.year, i.month, i.day) )
print ("读取当日时间格式n4",n4) #当日时间格式：yyyymmdd
n5 = ("%s" % i.isoformat() )
print ("读取当日时间格式n5",n5) #当日时间格式：yyyy-mm-dd:...
n8="20180101"#开始时间
print ("手动设定选股开始时间格式n8",n8)
n9="20180202"#结束时间
print ("手动设定选股结束时间格式n9",n9)
n10="20170101"#设定起始时间
print ("手动设定数据更新设定起始时间格式n10",n10)
x1="000001.XSHG"
print ("设定股票代码",x1)

#设定数据库连接：RQALPHA/TUSHARE
from funcat.data.tushare_backend import TushareDataBackend
from funcat.data.rqalpha_data_backend import RQAlphaDataBackend
backend = "rqalpha"
if backend == "rqalpha":
    set_data_backend(RQAlphaDataBackend("~/.rqalpha/bundle"))
elif backend == "tushare":
    set_data_backend(TushareDataBackend())
set_start_date(n10)

#当前股票最新数据显示。
S(x1)  # 设置当前关注股票
T(n9)   # 设置当前观察日期

print ("测试：打印 开盘 最高 最低 收盘 成交量",O, H, L, C, V)  # 打印 Open High Low Close
print ("当天涨幅",C / C[1] - 1)  # 当天涨幅
print ("打印60日均线",MA(C, 60))  # 打印60日均线
print ("判断收盘价是否大于60日均线",C > MA(C, 60))  # 判断收盘价是否大于60日均线
print ("30日最高价",HHV(H, 30))  # 30日最高价
print ("最近30日，收盘价 Close 大于60日均线的天数",COUNT(C > MA(C, 60), 30))  # 最近30日，收盘价 Close 大于60日均线的天数
print ("10日均线上穿",CROSS(MA(C, 10), MA(C, 20)))  # 10日均线上穿
print ("最近10天收阳线的天数",COUNT(C > O, 10))  # 最近10天收阳线的天数
print ("最近60天K线实体的最高价",HHV(MAX(O, C), 60))  # 最近60天K线实体的最高价
print ("5日均线上穿10日均线",CROSS(MA(C, 5), MA(C, 10)))  # 5日均线上穿10日均线


读取当日时间格式n1 2018-04-11
读取当日时间格式n2 20180411
读取前一日时间格式n3 2018-04-01
读取当日时间格式n4 2018411
读取当日时间格式n5 2018-04-11T09:24:08.998011
手动设定选股开始时间格式n8 20180101
手动设定选股结束时间格式n9 20180202
手动设定数据更新设定起始时间格式n10 20170101
设定股票代码 000001.XSHG
测试：打印 开盘 最高 最低 收盘 成交量 3419.22 3463.16 3388.86 3462.08 20812055100.0
当天涨幅 0.004380646246859632
打印60日均线 3381.247499999997
判断收盘价是否大于60日均线 True
30日最高价 3587.03
最近30日，收盘价 Close 大于60日均线的天数 23
10日均线上穿 False
最近10天收阳线的天数 6
最近60天K线实体的最高价 3563.64
5日均线上穿10日均线 False


In [2]:
print ("《选股》读取时间：==》",("%s" % i.isoformat() )) 
# 选出涨停股
select(
    lambda : C / C[1] - 1 >= 0.0995,
    start_date=(n3),
	end_date=(n2),
)

# 选出最近30天K线实体最高价最低价差7%以内，最近100天K线实体最高价最低价差25%以内，
# 最近10天，收盘价大于60日均线的天数大于3天
select(
    lambda : ((HHV(MAX(C, O), 30) / LLV(MIN(C, O), 30) - 1 < 0.07)
              & (HHV(MAX(C, O), 100) / LLV(MIN(C, O), 100) - 1 > 0.25)
              & (COUNT(C > MA(C, 60), 10) > 3)
             ),
    start_date=(n3),
	end_date=(n2),
)

# 选出最近3天每天的成交量小于20日成交量均线，最近3天最低价低于20日均线，最高价高于20日均线
# 自定义选股回调函数
def callback(date, order_book_id, symbol):
    print("Cool, 在", date, "选出", order_book_id, symbol)

    
select(
    lambda : (EVERY(V < MA(V, 20) / 2, 3) & EVERY(L < MA(C, 20), 3) & EVERY(H > MA(C, 20), 3)),
    start_date=(n3),
	end_date=(n2),
    callback=callback,
)


《选股》读取时间：==》 2018-04-11T09:24:08.998011
lambda : C / C[1] - 1 >= 0.0995,
[20180411]
No data for 000003.XSHE
No data for 000013.XSHE
No data for 000015.XSHE
No data for 000047.XSHE
No data for 000405.XSHE
No data for 000412.XSHE
No data for 000508.XSHE
No data for 000542.XSHE
No data for 000556.XSHE
No data for 000588.XSHE
No data for 000621.XSHE
No data for 000653.XSHE
No data for 000658.XSHE
No data for 000660.XSHE
No data for 000675.XSHE
No data for 000689.XSHE
No data for 000730.XSHE
No data for 002257.XSHE
No data for 002525.XSHE
No data for 002710.XSHE
No data for 002720.XSHE
No data for 300361.XSHE
No data for 300646.XSHE
No data for 300728.XSHE
No data for 300742.XSHE
No data for 300743.XSHE
No data for 600349.XSHG
No data for 600625.XSHG
No data for 600632.XSHG
No data for 600646.XSHG
No data for 600669.XSHG
No data for 600670.XSHG
No data for 600709.XSHG
No data for 600813.XSHG
No data for 600878.XSHG
No data for 601206.XSHG
No data for 603301.XSHG
No data for 603302.XSHG
No d

20180402 300582.XSHE 英飞特
20180402 300584.XSHE 海辰药业
20180402 300603.XSHE 立昂技术
20180402 300619.XSHE 金银河
20180402 300631.XSHE 久吾高科
20180402 300634.XSHE 彩讯股份
20180402 300648.XSHE 星云股份
20180402 300653.XSHE 正海生物
20180402 300655.XSHE 晶瑞股份
20180402 300661.XSHE 圣邦股份
20180402 300667.XSHE 必创科技
20180402 300670.XSHE 大烨智能
20180402 300682.XSHE 朗新科技
20180402 300684.XSHE 中石科技
20180402 300701.XSHE 森霸股份
20180402 300720.XSHE 海川智能
20180402 600207.XSHG 安彩高科
20180402 600718.XSHG 东软集团
20180402 600769.XSHG 祥龙电业
20180402 600929.XSHG 湖南盐业
20180402 601069.XSHG 西部黄金
20180402 603029.XSHG 天鹅股份
20180402 603067.XSHG 振华股份
20180402 603214.XSHG 爱婴室
20180402 603336.XSHG 宏辉果蔬
20180402 603363.XSHG 傲农生物
20180402 603386.XSHG 广东骏亚
20180402 603712.XSHG 七一二
20180402 603803.XSHG 瑞斯康达
20180402 603918.XSHG 金桥信息
20180402 603920.XSHG 世运电路

lambda : ((HHV(MAX(C, O), 30) / LLV(MIN(C, O), 30) - 1 < 0.07)
              & (HHV(MAX(C, O), 100) / LLV(MIN(C, O), 100) - 1 > 0.25)
              & (COUNT(C > MA(C, 60), 10) > 3)
             ),
